In [6]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download adrianmcmahon/imdb-india-movies

Dataset URL: https://www.kaggle.com/datasets/adrianmcmahon/imdb-india-movies
License(s): CC0-1.0
imdb-india-movies.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
from zipfile import ZipFile

rating='/content/imdb-india-movies.zip'
with ZipFile(rating, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')


The dataset is extracted


In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
Movie_rating = pd.read_csv('/content/IMDb Movies India.csv',encoding='latin-1')
Movie_rating

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
Movie_rating.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [19]:
Movie_rating.shape

(15509, 10)

In [20]:
Movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [21]:
Movie_rating.isnull().sum()

,0
Name,0
Year,528
Duration,8269
Genre,1877
Rating,7590
Votes,7589
Director,525
Actor 1,1617
Actor 2,2384
Actor 3,3144


In [22]:
Movie_rating.fillna(0,inplace=True)

In [25]:
Movie_rating.tail()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
15504,Zulm Ko Jala Doonga,(1988),0,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),0,Action,0.0,0,Kiran Thej,Sangeeta Tiwari,0,0
15507,Zulmi Shikari,(1988),0,Action,0.0,0,0,0,0,0
15508,Zulm-O-Sitam,(1998),130 min,"Action, Drama",6.2,20,K.C. Bokadia,Dharmendra,Jaya Prada,Arjun Sarja


In [83]:
label_encoder = LabelEncoder()

categorical_columns = ['Name', 'Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3','Year','Duration','Votes']

for col in categorical_columns:
    Movie_rating[col] = Movie_rating[col].astype(str)
    Movie_rating[col] = label_encoder.fit_transform(Movie_rating[col])


print(Movie_rating)

        Name  Year  Duration  Genre  Rating  Votes  Director  Actor 1  \
0          0     0         0     48       0      1      3223     3768   
1          1     5        95     48       7   1977      5014      792   
2       2083    37       165    118       0      1      3318     1386   
3       3608     5        35    434       4   1024       845      294   
4      11372    96        45     48       0      1      1584      563   
...      ...   ...       ...    ...     ...    ...       ...      ...   
15504   1141    68         0      1       5    375      5919     4556   
15505   1142    82        10    196       4   1776      5659     4136   
15506   1143    89         0      1       0      1      5555     1244   
15507   1144    68         0      1       0      1         0        0   
15508   1145    80        11    196       6    900      3013     4339   

       Actor 2  Actor 3  
0         3428      555  
1         2862     2964  
2          222     1025  
3         3790     

In [51]:
Movie_rating['Rating'] = Movie_rating['Rating'].round().astype(int)

In [53]:
x=Movie_rating.drop(columns=['Rating'],axis=1)
y=Movie_rating['Rating']

In [54]:
print(x)

        Name  Year  Duration  Genre  Votes  Director  Actor 1  Actor 2  \
0          0     0         0    225      1      1032     1393     4673   
1          1    99        10    225   1850       612     2537     4214   
2       8229   101       173    282      1      4585     3018     2076   
3       9464    99        11    145   1170      2581     2133      564   
4      10698    90         6    225      1      3179     2351     2739   
...      ...   ...       ...    ...    ...       ...      ...      ...   
15504   7465    68         0      1    369      1881     1765     3670   
15505   7466    79        30    346   1688      1670     1424     3928   
15506   7467    85         0      1      1      1586     2903        0   
15507   7468    68         0      1      1         0        0        0   
15508   7469    78        31    346    795      1331      159      658   

       Actor 3  
0         2346  
1         4298  
2         2726  
3         3360  
4         3402  
...      

In [55]:
print(y)

0        0
1        7
2        0
3        4
4        0
        ..
15504    5
15505    4
15506    0
15507    0
15508    6
Name: Rating, Length: 15509, dtype: int64


In [56]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [57]:
print(x.shape,x_train.shape,y_train.shape)

(15509, 9) (12407, 9) (12407,)


In [58]:
print(x_test)

       Name  Year  Duration  Genre  Votes  Director  Actor 1  Actor 2  Actor 3
8136    860    98         0    225      1      3157     1863     2805        0
12241   314    58         0    403      1         0        0        0        0
1000   2762    96       176    156   1526       257     3821     3883      601
3083   9130    62         0    225      1         0        0        0        0
14784  6664    30         0      0      1      2364     1301        0        0
...     ...   ...       ...    ...    ...       ...      ...      ...      ...
9790   2704    66         0    240    949      3407     3436     3319     4290
1330   4733    21         0     93      1       157     1176     3073     1469
13524  5225    28         0      0      1       224     1417        0        0
11673  7806    88        81    114    916       121     1853     1990     3680
13405  5094    86        32    424   1342      1482     1310     1977     3131

[3102 rows x 9 columns]


In [59]:
from sklearn.preprocessing import StandardScaler

In [60]:
scaler=StandardScaler()

In [61]:
x_train_std=scaler.fit_transform(x_train)

In [62]:
x_test_std=scaler.fit_transform(x_test)

In [63]:
import tensorflow as tf
tf.random.set_seed(3)
from tensorflow import keras

In [71]:
model=keras.Sequential([
                      keras.layers.Flatten(input_shape=(9,)),
                      keras.layers.Dense(64,activation='relu'),
                      keras.layers.Dense(35,activation='relu'),
                      keras.layers.Dense(1,activation='linear')
])

In [72]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

In [73]:
history=model.fit(x_train_std,y_train,validation_split=0.2,epochs=10)

Epoch 1/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 9.6496 - mean_absolute_error: 2.3493 - val_loss: 2.7919 - val_mean_absolute_error: 1.2315
Epoch 2/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.4883 - mean_absolute_error: 1.1440 - val_loss: 1.5559 - val_mean_absolute_error: 0.8854
Epoch 3/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5091 - mean_absolute_error: 0.8584 - val_loss: 1.3047 - val_mean_absolute_error: 0.7718
Epoch 4/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2968 - mean_absolute_error: 0.7685 - val_loss: 1.2205 - val_mean_absolute_error: 0.7324
Epoch 5/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2183 - mean_absolute_error: 0.7387 - val_loss: 1.1787 - val_mean_absolute_error: 0.7213
Epoch 6/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1735 - mean_absolute_error: 0.7230 - val_loss: 1.1550 - val_mean_absolute_error: 0.7205
Epoch 7/10
311/311 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1430 - mean_absolute_error: 0.7141 - v

In [74]:
loss,accuracy=model.evaluate(x_test_std,y_test)
print(accuracy)
print(f"test loss : {loss}")

97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1308 - mean_absolute_error: 0.6967
0.6900882124900818
test loss : 1.1180890798568726


In [75]:
y_pred=model.predict(x_test_std)

97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [76]:
print('Predicted price :',y_pred[:5])
print('Actual price :',y_test[:5])

Predicted price : [[-0.07380702]
 [-0.35055178]
 [ 6.739474  ]
 [-0.12078513]
 [ 0.18757364]]
Actual price : 8136     0
12241    0
1000     5
3083     0
14784    0
Name: Rating, dtype: int64


In [77]:
print(x_test_std)

[[-1.50353417  1.19736388 -0.61528613 ... -0.12674782  0.49824271
  -1.24797231]
 [-1.63991831 -0.22017018 -0.61528613 ... -1.42609834 -1.35417879
  -1.24797231]
 [-1.02843776  1.12648718  3.31250747 ...  1.2388605   1.21015372
  -0.8604086 ]
 ...
 [-0.41321039 -1.28332073 -0.61528613 ... -0.4378108  -1.35417879
  -1.24797231]
 [ 0.23149194  0.84298036  1.19239161 ... -0.13372233 -0.03998314
   1.12512994]
 [-0.44593259  0.77210366  0.09885816 ... -0.51243802 -0.04856834
   0.7710992 ]]


In [78]:
from google.colab import files
x_test.to_csv('rating_test',index=False)
files.download('rating_test')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [81]:
input_data=(8849,79,30,347,1951,3804,4190,2511,3537)
input_data_as_numpy_array=np.asarray(input_data)
input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)
input_data_std=scaler.transform(input_data_reshaped)
prediction=model.predict(input_data_std)

print("Rating of given movie is : ",prediction[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Rating of given movie is :  [5.436281]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
